In [3]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
from pymongo import MongoClient
import os
import base64
from dash.exceptions import PreventUpdate

# Import your custom CRUD Python module
from animal_shelter import AnimalShelter  # Modify if the filename or classname differs

# MongoDB connection setup
username = 'aacuser'
password = 'SNHU1234'
host = 'nv-desktop-services.apporto.com'
port = 31196
db_name = 'AAC'
client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}/{db_name}?authSource=admin')
db = client[db_name]

# Retrieve data from MongoDB
collection = db['animals']
documents = collection.find()  
df = pd.DataFrame(list(documents))

# Remove the MongoDB '_id' column
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
# Setup the JupyterDash application
app = JupyterDash('Animal Shelter Dashboard')

image_path = '/home/eriksierra_snhu/Desktop/Grazioso Salvare Logo.png'

#checks if image exists
if os.path.exists(image_path):
    encoded_image = open(image_path, 'rb').read()
    logo = f"data:image/png;base64,{base64.b64encode(encoded_image).decode()}"
else:
    logo = None
    
# Define the app layout
app.layout = html.Div([
    html.Div([
        html.Img(src=logo, style={'height': '100px', 'width': 'auto', 'float': 'left', 'margin-right': '10px'}),
        html.H1('AAC Outcomes Dashboard - Erik Sierra', style={'display': 'inline-block', 'vertical-align': 'top', 'margin': '20px 0px'}),
    ]),
    # Filter Options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin': '5px'}
        )
    ], style={'text-align': 'center', 'margin': '10px'}),
    
    # Data Table and Map + Pie Chart Row
    html.Div([
        html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records'),
                editable=False,
                filter_action="native",
                sort_action="native",
                page_action="native",
                page_current=0,
                page_size=10,
                row_selectable="single",
            )
        ], className='six columns'),
        
        # map and pie chart
        html.Div([
            html.Div([
                dcc.Graph(id='pie-chart')
            ], className='six columns'),

            html.Div([
                dl.Map(style={'width': '1000px', 'height': '500px'},
                       id='map',
                       center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id")
                ]),
            ], className='six columns'),
        ], className='row') 
    ])
])


@app.callback(
    Output('map', 'children'),  # Change 'map-id' to 'map'
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        # Return a marker at the default location if no selection is made
        return [
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip("No selection"),
                dl.Popup([
                    html.H1("No Animal Selected"),
                    html.P("Select an animal to see its location.")
                ])
            ])
        ]

    # Convert viewData to DataFrame
    dff = pd.DataFrame(viewData)
    # Check if the selected_rows index list is empty or if the selected row is beyond the DataFrame length
    if not selected_rows or selected_rows[0] >= len(dff):
        # Return only the tile layer if no valid selection is made
        return [dl.TileLayer(id="base-layer-id")]

    # Ensure you're accessing valid row indexes
    row = selected_rows[0]
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        latitude = dff.loc[row, 'location_lat']
        longitude = dff.loc[row, 'location_long']
        # Update and return the marker on the map to the selected animal's location
        return [
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(dff.loc[row, 'breed']),
                dl.Popup([
                    html.H1(dff.loc[row, 'name']),
                    html.P(dff.loc[row, 'breed'])
                ])
            ])
        ]
    else:
        # Return only the tile layer if latitude or longitude is missing
        return [dl.TileLayer(id="base-layer-id")]


# A dictionary to map the rescue type to the preferred breeds, sex, and age range
RESCUE_CRITERIA = {
    'Water': {
        'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex': 'Intact Female',
        'age_range': (26, 156)
    },
    'Mountain': {
        'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_range': (26, 156)
    },
    'Disaster': {
        'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_range': (20, 300)
    }
}

from dash.exceptions import PreventUpdate

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(rows):
    if not rows:
        raise PreventUpdate

    dff = pd.DataFrame(rows)
    pie_data = dff['breed'].value_counts()  # Assuming 'breed' is the column you want to display in the pie chart

    fig = {
        'data': [{
            'values': pie_data.values,
            'labels': pie_data.index,
            'type': 'pie',
            'hoverinfo': 'label+percent',
        }],
        'layout': {
            'title': 'Distribution of Breeds',
            'showlegend': True
        }
    }

    return fig


# Callback for updating the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_value):
    # Create the initial query
    query = {}
    if filter_value in RESCUE_CRITERIA:
        # Get the criteria for the selected rescue type
        criteria = RESCUE_CRITERIA[filter_value]
        
        # Construct the query based on the criteria
        query = {
            'breed': {'$in': criteria['breeds']},
            'sex_upon_outcome': criteria['sex'],
            'age_upon_outcome_in_weeks': {'$gte': criteria['age_range'][0], '$lte': criteria['age_range'][1]}
        }
    
    # Query the database and update the table data
    filtered_documents = collection.find(query)
    df_filtered = pd.DataFrame(list(filtered_documents))

    # Remove the MongoDB '_id' column
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')



if __name__ == '__main__':
    app.run_server(mode='inline')
